In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10
# Combine the features and labels of the training data
dataset = gdata.ArrayDataset(features, labels)
# Randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for X, y in data_iter:
    print(X, y)
    break


[[ 1.0691297  -0.84440124]
 [-0.70166826  0.18638176]
 [-1.4493372  -0.45174706]
 [-0.25344443  0.35584167]
 [ 0.5817126   0.08419139]
 [-1.1652131   0.36885762]
 [-2.0051894  -1.0490869 ]
 [ 1.2168038  -0.5497378 ]
 [-0.29304633 -1.0861639 ]
 [ 0.5035904  -1.189445  ]]
<NDArray 10x2 @cpu(0)> 
[9.2099905 2.1595848 2.8295398 2.4974496 5.079607  0.6253812 3.743934
 8.521942  7.3145    9.243224 ]
<NDArray 10 @cpu(0)>


In [4]:
from mxnet.gluon import nn
net = nn.Sequential()

In [5]:
net.add(nn.Dense(1))

In [6]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

In [7]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss() # The squared loss is also known as the L2 norm loss

In [8]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

In [9]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print(f'epoch {epoch}, loss {l.mean().asnumpy()}')

epoch 1, loss [0.03500612]
epoch 2, loss [0.00012078]
epoch 3, loss [4.877244e-05]


In [11]:
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)

Error in estimating w 
[[ 0.00086212 -0.00034618]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[0.00030422]
<NDArray 1 @cpu(0)>
